# Plot spot position as time series data
env : data_vis_32

# 1.0 Import relevant packages

In [1]:
# import pypyodbc
import pandas as pd
import plotly.express as px
import seaborn as sns
from matplotlib.colors import to_hex

# 2.0 Import spot position and size QA data

In [ ]:
data_path = r"../data/xlsx_exported_from_access/SpotPositionResults.xlsx"

df = pd.read_excel(data_path)

df.head(2)



,ADate,MachineName,Energy,Device,Gantry Angle,Spot,x-pos,y-pos,hor_rt_gradient,hor_lt_gradient,hor_fwhm,vert_rt_gradient,vert_lt_gradient,vert_fwhm,bltr_rt_gradient,bltr_lt_gradient,bltr_fwhm,tlbr_rt_gradient,tlbr_lt_gradient,tlbr_fwhm
0,2022-04-21 16:44:14,Gantry 2,70,XRV-3000,180,Bottom-Centre,-0.2357,124.9224,-9.059840,9.157258,13.342140,-8.964427,9.333333,13.536155,-8.485281,8.747554,14.216962,-8.909545,8.992812,13.842831
1,2022-04-21 16:44:14,Gantry 2,70,XRV-3000,180,Bottom-Left,-125.0153,125.4501,-8.871094,9.120482,13.562307,-8.964427,9.333333,13.712522,-8.747554,8.591347,14.310494,-8.747554,8.992812,13.842831


# 3.0 exploratory data analysis - understand your data

In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41172 entries, 0 to 41171
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ADate             41172 non-null  datetime64[ns]
 1   MachineName       41172 non-null  object        
 2   Energy            41172 non-null  int64         
 3   Device            41172 non-null  object        
 4   Gantry Angle      41172 non-null  int64         
 5   Spot              41172 non-null  object        
 6   x-pos             41172 non-null  float64       
 7   y-pos             41172 non-null  float64       
 8   hor_rt_gradient   41172 non-null  float64       
 9   hor_lt_gradient   41172 non-null  float64       
 10  hor_fwhm          41172 non-null  float64       
 11  vert_rt_gradient  41172 non-null  float64       
 12  vert_lt_gradient  41172 non-null  float64       
 13  vert_fwhm         41172 non-null  float64       
 14  bltr_rt_gradient  4117

In [5]:
df.value_counts("MachineName"), df.value_counts("Device"), df.value_counts("Energy")

(MachineName
 Gantry 3    10576
 Gantry 1    10472
 Gantry 4    10357
 Gantry 2     9767
 dtype: int64,
 Device
 XRV-3000    31815
 XRV-4000     9357
 dtype: int64,
 Energy
 150    8250
 240    8243
 200    8233
 100    8232
 70     8214
 dtype: int64)

# 4.0 filtering data

In [9]:
sub_df = df[["ADate",	"MachineName", 	"Energy", "Device", "Gantry Angle", "Spot", "x-pos", "y-pos"]].copy()

## calculate abs shift

In [17]:
pred_xrv4000 = {'Top-Top-Left': [-125, -175], 'Top-Top-Centre': [0, -175], 'Top-Top-Right': [125, -175], \
                'Top-Left': [-125, -125], 'Top-Centre':[0, -125], 'Top-Right':[125, -125], \
                'Left': [-125, 0], 'Centre':[0, 0], 'Right':[125, 0], \
                'Bottom-Left': [-125, 125], 'Bottom-Centre':[0, 125], 'Bottom-Right':[125, 125], \
                'Bottom-Bottom-Left': [-125, 175], 'Bottom-Bottom-Centre': [0, 175], 'Bottom-Bottom-Right': [125, 175]}

sub_df['px_pos'] = sub_df['Spot'].map(lambda s: pred_xrv4000[s][0] if s in pred_xrv4000 else None)
sub_df['py_pos'] = sub_df['Spot'].map(lambda s: pred_xrv4000[s][1] if s in pred_xrv4000 else None)

sub_df['abs_xpos'] = sub_df["x-pos"] - sub_df["px_pos"]
sub_df['abs_ypos'] = sub_df["y-pos"] - sub_df["py_pos"]

In [18]:
print(sub_df.head(2))

                ADate MachineName  Energy    Device  Gantry Angle  \
0 2022-04-21 16:44:14    Gantry 2      70  XRV-3000           180   
1 2022-04-21 16:44:14    Gantry 2      70  XRV-3000           180   

            Spot     x-pos     y-pos  px_pos  py_pos  abs_xpos  abs_ypos  
0  Bottom-Centre   -0.2357  124.9224       0     125   -0.2357   -0.0776  
1    Bottom-Left -125.0153  125.4501    -125     125   -0.0153    0.4501  


In [ ]:

def plotly_spot_position(df, pos, gantry, device, energy, gantry_angle, n_months):
    """ plot spot position time series data
        df = dataframe
        gantry = "Gantry 1", "Gantry 2", 
        pos = "abs_xpos",
        device = "XRV-3000", "XRV-4000"
        energy = int,
        gantry_angle = 0,90,180,270
        n_month = int

    
     """
    
     # only show data from last 12 months
    start_date = pd.Timestamp.today() - pd.DateOffset(months=n_months)

    selected_df = df[(df["MachineName"]==gantry) & (df["Device"] == device) & (df['ADate'] >= start_date) & (df["Energy"] == energy) &(df["Gantry Angle"] == gantry_angle)]

    # set colour
    palette = sns.color_palette("deep", n_colors=df['Spot'].nunique())
    palette_hex = [to_hex(c) for c in palette]


    # Plot
    fig = px.scatter(
        selected_df,
        x='ADate',
        y=pos,
        symbol='Spot', 
        color='Spot',        # hue
         color_discrete_sequence= px.colors.qualitative.T10,
        title=f'{gantry} - absolute shift- {pos}',
        labels={'x-pos': 'X Position', 'adate': 'Date'},
        height=500
    )

    # Add tolerance bands +/- 2
    fig.add_hline(y=2, line_dash="dash", line_color="grey", annotation_text="tolerance", annotation_position="top left")
    fig.add_hline(y=-2, line_dash="dash", line_color="grey")



    # Optional: connect points by spot for clarity
    fig.update_traces(mode='markers+lines',
                    marker=dict(size=12,               # larger size
                                line=dict(width=2)),     # outline width
                    line=dict(width=1                # thinner connecting lines
                    ))

    # Show plot
    fig.show()

    
    return 


# plotting absolute y-pos, on Gantry 4, from XRV-4000 data, 70 MeV spot, Gantry angle = 0, in last2 months
plotly_spot_position(sub_df, "abs_ypos", "Gantry 4", "XRV-4000", 70, 0, 24)


In [20]:
plotly_spot_position(sub_df, "abs_ypos", "Gantry 4", "XRV-4000", 70, 0, 24)

## plot another device

In [8]:
plotly_spot_position(sub_df, "abs_xpos", "Gantry 4", "XRV-3000", 70, 0, 24)

In [23]:
start_date = pd.Timestamp.today() - pd.DateOffset(months=12)
selected_df = sub_df[(df["MachineName"]=="Gantry 2") & (df["Device"] == "XRV-3000") & (df['ADate'] >= start_date)].copy()
# Calculate average abs_xpos per adate and energy
selected_df['avg_abs_pos'] = selected_df.groupby(['ADate', 'Energy', "Gantry Angle"])["abs_xpos"].transform('mean')

selected_df.head(5)

,ADate,MachineName,Energy,Device,Gantry Angle,Spot,x-pos,y-pos,px_pos,py_pos,abs_xpos,abs_ypos,avg_abs_pos
29630,2025-01-18 09:39:22,Gantry 2,70,XRV-3000,0,Bottom-Centre,0.4206,125.2846,0,125,0.4206,0.2846,0.337467
29631,2025-01-18 09:39:22,Gantry 2,70,XRV-3000,0,Bottom-Left,-124.7152,125.1824,-125,125,0.2848,0.1824,0.337467
29632,2025-01-18 09:39:22,Gantry 2,70,XRV-3000,0,Bottom-Right,125.7485,125.1006,125,125,0.7485,0.1006,0.337467
29633,2025-01-18 09:39:22,Gantry 2,70,XRV-3000,0,Centre,0.2611,0.1502,0,0,0.2611,0.1502,0.337467
29634,2025-01-18 09:39:22,Gantry 2,70,XRV-3000,0,Left,-124.9685,0.5777,-125,0,0.0315,0.5777,0.337467


In [28]:

def plotly_ave_spot_position(df, parameter, gantry, device,  n_months):
    """ plot average spot position across all spot positions with the same adate and energy
        df = dataframe
        gantry = "Gantry 1", "Gantry 2", 
        paramter = "abs_xpos",
        device = "XRV-3000", "XRV-4000"
        energy = int
        n_month = int

    
     """
    
     # only show data from last 12 months
    start_date = pd.Timestamp.today() - pd.DateOffset(months=n_months)

    selected_df = df[(df["MachineName"]==gantry) & (df["Device"] == device) & (df['ADate'] >= start_date)].copy()


    # Calculate average abs_xpos per adate and energy
    selected_df['avg_abs_pos'] = selected_df.groupby(['ADate', 'Energy', "Gantry Angle"])[parameter].transform('mean')

    # want to displace energy as discrete colour not spectrum
    selected_df['Energy'] = df['Energy'].astype(int).astype(str)

    

    # Plot
    fig = px.scatter(
        selected_df,
        x='ADate',
        y='avg_abs_pos',
        symbol='Gantry Angle', 
        color='Energy',        # hue
        title=f'average {parameter} across all spot positions with the same adate and energy',
        labels={'x-pos': 'X Position', 'adate': 'Date'},
        height=500
    )

    # Add tolerance bands +/- 2
    fig.add_hline(y=2, line_dash="dash", line_color="grey", annotation_text="tolerance", annotation_position="top left")
    fig.add_hline(y=-2, line_dash="dash", line_color="grey")



    # Optional: connect points by spot for clarity
       # Optional: connect points by spot for clarity
    fig.update_traces(mode='markers+lines',
                    marker=dict(size=12,               # larger size
                                line=dict(width=2)),     # outline width
                    line=dict(width=1                # thinner connecting lines
                    ))

    # Show plot
    fig.show()

    
    return 




In [29]:
plotly_ave_spot_position(sub_df, "abs_xpos", "Gantry 1", "XRV-3000",  24)